Here we attempt to take a demographic score across postcodes (D_pc in the range [0,dmax]) and map it to a target range (T_pc in [tmin,tmax]) but also has the property that its weighted sum over postcodes gives a particular value for that state (T_state), i.e., if the set of weights for volume weighting over postcodes are f_pc st. sum_pc[f_pc] == 1, then sum_pc[(f_pc * T_pc)] == T_state 

In [515]:
import numpy as np
from scipy.optimize import fsolve, minimize, broyden1, broyden2, basinhopping, shgo
import plotly.graph_objects as go
from numpy import linalg as LA

In [516]:
sales = np.array([100, 200, 500, 250, 500, 6000,
                 5000, 20000])  # dummy sales data
f = sales / sum(sales)
D = np.array([0, 12, 47, 520, 62, 33, 400, 900])  # dummy demographic score
T_state = 0.5  # state level target
tmin = 0.25
tmax = 0.75

use_real_dataset = True
if use_real_dataset:
    D2 = np.array(D2)
    f2 = np.array(f2)
    D = D2 # D2 and f2 are copy and pasted from PG's excel workbook, but with zero population postcodes filtered out
    f2 = f2 / sum(f2) # correct for what is likely the filtering out of zero population postcodes such that the sum is slightly less than 1
    f = np.array(f2)
    print(f'Check on sum of f2: {sum(f)}')
    # temper the low statistic

Check on sum of f2: 1.0


In [517]:
X = (D - min(D)) / (max(D) - min(D))  # normalize demographic score to [0, 1]

In [643]:
if use_real_dataset:
    test_z = [1, 3]
    test_z = z

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X, y=f2, mode="markers", name="f2 vs D2", marker_color=f))
    fig.add_trace(go.Scatter(x=T0(test_z), y=f2+max(f2)*1.3, mode="markers", name="f2 vs D2 mod", marker_color=f))

    fig.show()

    fig2 = go.Figure()

    fig2.add_trace(
        go.Histogram(
            x=X,
            xbins=dict(start=0, end=1, size=0.005),
            name="Without linear modifier function",
        )
    )
    fig2.add_trace(
        go.Histogram(
            x=T1(test_z),
            xbins=dict(start=0, end=1, size=0.005),
            name="With linear modifier function",
        )
    )

    fig2.update_layout(barmode="overlay")
    # Reduce opacity to see both histograms
    fig2.update_traces(opacity=0.5)
    fig2.update_xaxes(range=[0, 1])
    #fig2.update_yaxes(range=[0, 200])
    fig2.show()

    fig3 = go.Figure()
    tmp_x = np.linspace(0, 1, 100)
    fig3.add_trace(go.Scatter(x=tmp_x, y=Y(test_z,tmp_x)))
    fig3.show()

In [579]:
xx = np.linspace(0, 1, 100)
y2 = 0.1*abs(np.power(abs(xx-1),2))
#y2 = max(y2) - y2
y3 = 0.2*abs(np.power(abs(xx-1),3.3))
fig = go.Figure()
fig.add_trace(go.Scatter(x=xx, y=y2))
fig.add_trace(go.Scatter(x=xx, y=y3))
fig.show() 

In [580]:
def Y1(z, X):
    m = z[0]
    b = z[1]
    return (
        m * X + b
    )  # this is a linear function by which we will transform the demographic normalized score

def Y2(z, X):
    m = z[0]
    b = z[1]
    return (
        m * np.power(np.abs(X-1),b)
    )  # this is a non-linear function by which we will transform the demographic normalized score


def Y(z, X):
    return Y2(z, X)

def T0(z):
    return X - Y(z, X) 

def T1(z):
    XY = T0(z)
    return (XY - min(XY)) / (max(XY) - min(XY))

def T(z):
    return T1(z) * (tmax - tmin) + tmin

In [581]:
def eq1(z):
    res = sum(f * T(z)) - T_state
    return res

def eq2(z):
    m = z[0]
    b = z[1]
    return m * max(X) + b - 1

def my_function(z):
    return eq1(z), eq2(z)

def my_scalar_function(z):
    return LA.norm(my_function(z))

def abs_eq1(z):
    return np.abs(eq1(z))

In [582]:
np.power(np.array([-1.2,-2,0.0,2,3,4,5]),2)
np.float_power(-1.2,2.3)


/var/folders/rd/12c6t5q92pv9swq7rhsy8m3w0000gr/T/ipykernel_53075/1326130933.py:2: RuntimeWarning:

invalid value encountered in float_power



nan

In [608]:
if use_real_dataset:
    z_guess = [1,2]
else:
    z_guess = [-0.1,2]

# try local minimization
print('Local MINIMIZATION')
linear_modifer = False
if linear_modifer:
    #cons = ({'type': 'eq', 'fun': lambda x:  (x[0]*1+x[1]-1)})    
    #cons = ({'type': 'ineq', 'fun': lambda x:  (x[0]*1+x[1]-1)})    
    bnds = ((-1,  1), (-1, 1))
else:
    bnds = ((0.1,  1e6), (2, 40))
print(f'bnds: {bnds}')
print(f'z_guess: {z_guess}')
#zz = minimize(abs_eq1, z_guess, constraints=cons, bounds=bnds, method = 'SLSQP', tol=1e-8)
#zz = minimize(abs_eq1, z_guess, bounds=bnds, method = 'SLSQP', tol=1e-8)
zz = minimize(abs_eq1, z_guess, bounds = bnds, method='Nelder-Mead', tol=1e-8)

print(zz)
print(zz.x)
for i in np.linspace(bnds[0][0],bnds[0][1],10):
    for j in np.linspace(bnds[1][0],bnds[1][1],10):
        #print(f'[{i},{j}]:{np.power(np.abs(X-1),j)}')
        #print(f'[{i},{j}]:{np.power(np.abs(np.array([-1,0,1])),j)}')
        #print(f'Y([{i},{j}]: {Y([i,j],X)}')
        #print(f'T1([{i},{j}]: {T1([i,j])}')
        print(f'abs_eq1([{i},{j}]: {abs_eq1([i,j])}')


print(f'value of line at x=1: {Y(zz.x, 1)}')

# try non-linear solver
print('Nonlinear FSOLVE')
z, infodict, ier, mesg = fsolve(my_function, z_guess, full_output=True)
print(f"Solved for m={z[0]}, b={z[1]}")
print(f"Solver return code: {ier}")
print(f"Solver mesg: {mesg}")
print(f"Residual for eq1 with linear modification applied: {eq1(z)}")
print(f"Residual for eq2 with linear modification applied: {eq2(z)}")
print(f"Residual for eq1 without linear modification applied (m=0,b=1): {eq1([0, 1])}")
print(f"Residual for eq2 without linear modification applied (m=0,b=1): {eq2([0, 1])}")

# try Broyden's method
print('Global MINIMIZATION')

zb = shgo(abs_eq1, bnds, constraints=cons)
print(zb)

z = zz.x
T_with_solve = T(z)
T_without_solve = T([0, 1])

Local MINIMIZATION
bnds: ((0.1, 1000000.0), (2, 40))
z_guess: [1, 2]
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 5.551115123125783e-17
             x: [ 2.854e+00  2.223e+01]
           nit: 97
          nfev: 194
 final_simplex: (array([[ 2.854e+00,  2.223e+01],
                       [ 2.854e+00,  2.223e+01],
                       [ 2.854e+00,  2.223e+01]]), array([ 5.551e-17,  3.331e-16,  5.551e-16]))
[ 2.8538504  22.22622793]
abs_eq1([0.1,2.0]: 0.19500036014983113
abs_eq1([0.1,6.222222222222222]: 0.18546803744945956
abs_eq1([0.1,10.444444444444445]: 0.18046555587720803
abs_eq1([0.1,14.666666666666668]: 0.17743926338692906
abs_eq1([0.1,18.88888888888889]: 0.17542354549383843
abs_eq1([0.1,23.11111111111111]: 0.17398138158184023
abs_eq1([0.1,27.333333333333336]: 0.17289194572349487
abs_eq1([0.1,31.555555555555557]: 0.1720337801938711
abs_eq1([0.1,35.77777777777778]: 0.17133529487234322
abs_eq1([0.1,40.0]: 0.170751803654

In [609]:
print(D)
print(X)
print(T_with_solve)
print(T_without_solve)
print(sorted(T_with_solve))
print(sorted(T_without_solve))
print(f'State target with solve: {sum(f * T_with_solve)}')
print(f'State target without solve: {sum(f * T_without_solve)}')
print(f'Min T_with_solve: {min(T_with_solve)}')
print(f'Max T_with_solve: {max(T_with_solve)}')
print(f'Min T_without_solve: {min(T_without_solve)}')
print(f'Max T_without_solve: {max(T_without_solve)}')

[2364 1750 1860 ...    1    1 6733]
[0.1894644  0.14023412 0.14905388 ... 0.         0.         0.53976908]
[0.64136661 0.62557039 0.62935336 ... 0.25       0.25       0.69028946]
[0.3447322  0.32011706 0.32452694 ... 0.25       0.25       0.51988454]
[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.2

In [610]:
# plot up the linear modifier function

import plotly.graph_objects as go

x = np.linspace(0, 1, 100)
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=x, y=Y(z, x), mode="lines",
               name="Solved for modifier function")
)
fig.add_trace(
    go.Scatter(
        x=x, y=Y(z_guess, x), mode="lines", name="Initial guess for modifer function"
    )
)

fig.show()

In [638]:
H1 = np.histogram(T_without_solve,bins=200)
H2 = np.histogram(T_with_solve,bins=200)
fig2 = go.Figure()
fig2.add_trace(go.Bar(x=H2[1], y=(H2[0])))
fig2.add_trace(go.Bar(x=H1[1], y=(H1[0])))
fig2.update_yaxes(range=[0, 100])
fig2.show()
print(H1)

(array([1168,  117,   73,   55,   33,   27,   19,   26,   18,   18,   22,
         14,   21,   22,   20,   13,   15,   18,   13,   16,   15,   19,
         21,   18,    9,   20,   24,   18,   13,   18,   18,   11,   11,
         14,   13,    9,   15,   17,   16,    9,   17,   13,   12,   16,
          8,   10,    9,   11,    8,    5,    4,    6,    4,    5,   10,
          5,    3,    5,    3,    2,    4,    4,    3,    1,    2,    4,
          2,    3,    0,    1,    1,    2,    3,    2,    3,    0,    0,
          1,    1,    0,    0,    1,    0,    1,    0,    0,    2,    1,
          1,    1,    0,    1,    1,    1,    3,    2,    1,    2,    0,
          0,    0,    0,    1,    0,    0,    1,    3,    3,    0,    0,
          1,    0,    0,    1,    0,    0,    1,    0,    1,    1,    1,
          0,    1,    0,    1,    1,    0,    0,    0,    0,    0,    0,
          1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1,    0,    0,    0,    0,    0,    0,  

In [611]:
# plot the transformed demographic score

import plotly.graph_objects as go

fig = go.Figure()

if use_real_dataset:
    fig.add_trace(go.Histogram(x=T_with_solve, xbins=dict(start=0, end=1, size=0.005), name="With linear modifier function"))
    fig.add_trace(go.Histogram(x=T_without_solve, xbins=dict(start=0, end=1, size=0.005), name="Without linear modifier function"))

    fig.update_layout(barmode="overlay")
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.75)
    fig.update_xaxes(range=[0, 1])
    fig.update_yaxes(range=[0, 200])

else:
    fig.add_trace(
        go.Scatter(
            x=T(z),
            y=X * 0,
            mode="markers",
            marker=dict(size=30),
            name=f"Transformed with linear modifer function ({sum(f*T_with_solve)})",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=T([0, 1]),
            y=X * 0,
            mode="markers",
            marker=dict(size=10),
            name=f"Transformed without linear modifer function ({sum(f*T_without_solve)})",
        )
    )

fig.show()

In [612]:
np.log10([1,10,100])

array([0., 1., 2.])

In [398]:
D2 = [
    2364,
    1750,
    1860,
    1305,
    1622,
    5,
    65,
    52,
    1534,
    1662,
    63,
    48,
    57,
    3,
    3,
    63,
    3,
    2,
    2,
    83,
    237,
    14,
    236,
    103,
    282,
    1,
    3,
    12229,
    12473,
    3835,
    3856,
    3953,
    7074,
    7463,
    5798,
    8259,
    141,
    3462,
    4051,
    5494,
    6865,
    4677,
    5234,
    1036,
    8962,
    39,
    3789,
    893,
    5985,
    1322,
    4623,
    2173,
    3892,
    4608,
    4922,
    6657,
    6607,
    3645,
    4655,
    6654,
    7365,
    5882,
    7,
    4171,
    1410,
    1390,
    1415,
    2074,
    1923,
    1922,
    1684,
    1032,
    1226,
    2159,
    2968,
    645,
    638,
    144,
    141,
    102,
    53,
    4,
    1476,
    7764,
    5854,
    2700,
    2339,
    3380,
    3382,
    3063,
    743,
    2730,
    47,
    10,
    164,
    173,
    2331,
    1719,
    1435,
    2327,
    2450,
    3291,
    355,
    2352,
    3407,
    2554,
    2554,
    2969,
    69,
    1279,
    1816,
    991,
    1596,
    2281,
    5567,
    3710,
    36,
    934,
    4389,
    146,
    2970,
    3009,
    5376,
    3567,
    2719,
    2845,
    4833,
    2371,
    1885,
    1430,
    2837,
    946,
    2621,
    3375,
    2540,
    2277,
    489,
    60,
    60,
    29,
    1753,
    3622,
    2908,
    2743,
    3035,
    4184,
    3534,
    3333,
    1334,
    2318,
    286,
    286,
    76,
    2590,
    3302,
    97,
    60,
    50,
    4542,
    4547,
    2740,
    7538,
    4041,
    2698,
    2701,
    4068,
    4176,
    6660,
    73,
    164,
    904,
    1669,
    3886,
    3324,
    2656,
    3452,
    10,
    1,
    5106,
    4454,
    3236,
    3582,
    1196,
    4104,
    715,
    931,
    2715,
    2498,
    2502,
    3136,
    3537,
    3144,
    1443,
    980,
    2734,
    1485,
    900,
    936,
    861,
    834,
    849,
    52,
    619,
    1135,
    1401,
    774,
    1575,
    622,
    57,
    721,
    101,
    906,
    476,
    1217,
    452,
    855,
    1116,
    101,
    989,
    1931,
    1207,
    2018,
    1907,
    2676,
    2655,
    300,
    2676,
    2676,
    2542,
    1661,
    3326,
    2715,
    1908,
    2058,
    1569,
    1532,
    1021,
    1024,
    4,
    8,
    43,
    463,
    104,
    463,
    55,
    231,
    58,
    114,
    69,
    1,
    90,
    269,
    230,
    238,
    1,
    1,
    107,
    137,
    42,
    35,
    31,
    1,
    38,
    1,
    2,
    169,
    2,
    1,
    1,
    2,
    2,
    2,
    2,
    78,
    88,
    2,
    2,
    1,
    2,
    1,
    2,
    1,
    47,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    97,
    1,
    1,
    31,
    1,
    1,
    94,
    1,
    1,
    1,
    1,
    1,
    2,
    4,
    8,
    2,
    5,
    4,
    2,
    4,
    4,
    363,
    50,
    122,
    124,
    36,
    56,
    3,
    30,
    153,
    36,
    56,
    44,
    175,
    6,
    407,
    171,
    2,
    18,
    45,
    74,
    118,
    2,
    21,
    25,
    25,
    25,
    9,
    72,
    4,
    11,
    11,
    4,
    1,
    3,
    46,
    387,
    40,
    247,
    231,
    34,
    145,
    195,
    207,
    1159,
    381,
    75,
    75,
    52,
    2563,
    1175,
    1382,
    1129,
    73,
    739,
    1305,
    1307,
    1514,
    1759,
    1224,
    1224,
    609,
    496,
    1936,
    921,
    395,
    628,
    36,
    17,
    197,
    79,
    258,
    281,
    146,
    195,
    2,
    39,
    117,
    92,
    26,
    33,
    115,
    103,
    41,
    878,
    878,
    804,
    442,
    1517,
    40,
    40,
    172,
    79,
    104,
    104,
    89,
    101,
    235,
    61,
    5,
    3,
    301,
    2,
    36,
    1,
    2,
    1,
    1,
    2,
    5,
    5,
    20,
    852,
    2685,
    1639,
    1372,
    3019,
    1372,
    1584,
    1545,
    26,
    1735,
    2960,
    1862,
    1675,
    1565,
    23,
    935,
    1040,
    1,
    2,
    2,
    1,
    2,
    2,
    1,
    50,
    1,
    1,
    1,
    247,
    715,
    2,
    2,
    2,
    2,
    59,
    2,
    14,
    1,
    432,
    2,
    23,
    1,
    2,
    2,
    2,
    2,
    2,
    215,
    3,
    1,
    2,
    1,
    1,
    1,
    1,
    108,
    291,
    1,
    1,
    2,
    2,
    28,
    25,
    1,
    1,
    36,
    2,
    2,
    2,
    2,
    21,
    130,
    1,
    2,
    5,
    2,
    2,
    1,
    1,
    39,
    20,
    21,
    1655,
    1737,
    78,
    1601,
    101,
    144,
    191,
    208,
    354,
    68,
    60,
    2287,
    1979,
    2735,
    681,
    3223,
    50,
    1561,
    2372,
    1728,
    2509,
    642,
    642,
    14,
    314,
    314,
    245,
    245,
    323,
    300,
    245,
    245,
    41,
    35,
    2,
    84,
    5,
    4,
    2,
    34,
    36,
    5,
    4,
    5,
    282,
    1,
    2,
    2,
    2,
    1,
    2,
    1,
    1,
    1,
    2,
    2,
    1,
    1,
    1,
    1,
    395,
    1,
    1,
    1,
    1,
    3,
    3,
    3,
    1,
    2,
    128,
    1,
    3,
    3,
    3,
    3,
    2,
    1,
    43,
    2,
    1,
    1,
    100,
    28,
    57,
    818,
    1390,
    1465,
    1403,
    1452,
    1702,
    2871,
    1912,
    2425,
    2506,
    118,
    1684,
    2642,
    6733,
    1843,
    4106,
    2672,
    2816,
    2141,
    2201,
    776,
    2608,
    1591,
    2912,
    1629,
    1980,
    2,
    947,
    5,
    314,
    1759,
    889,
    1033,
    5383,
    3446,
    489,
    2033,
    1197,
    3807,
    149,
    1865,
    2094,
    1560,
    1260,
    4,
    342,
    1321,
    2758,
    1290,
    1843,
    1843,
    9899,
    3537,
    4473,
    5868,
    435,
    245,
    2751,
    873,
    214,
    791,
    7838,
    7631,
    5459,
    4088,
    25,
    3804,
    2959,
    1341,
    5,
    1385,
    2880,
    2322,
    2283,
    2182,
    1923,
    2250,
    1725,
    1194,
    2439,
    1636,
    538,
    324,
    324,
    1383,
    1383,
    1177,
    537,
    108,
    173,
    5,
    1584,
    2919,
    2413,
    1713,
    1064,
    2414,
    2915,
    2866,
    806,
    192,
    806,
    367,
    544,
    119,
    4200,
    2934,
    2639,
    2792,
    2960,
    3394,
    2960,
    2743,
    2569,
    2566,
    2153,
    1910,
    2259,
    1974,
    1310,
    1881,
    47,
    190,
    775,
    3034,
    4355,
    3345,
    3009,
    3097,
    2892,
    2990,
    2370,
    2382,
    2601,
    1493,
    1414,
    798,
    2107,
    1534,
    1094,
    178,
    393,
    3589,
    3843,
    4571,
    3390,
    2789,
    2091,
    1778,
    1886,
    2037,
    2165,
    33,
    3464,
    1010,
    2341,
    1638,
    1095,
    1094,
    6385,
    137,
    5943,
    5700,
    4521,
    2800,
    3035,
    3229,
    788,
    2506,
    2712,
    2736,
    2468,
    3003,
    2431,
    2554,
    2089,
    1406,
    1658,
    1480,
    1103,
    788,
    4205,
    3402,
    5867,
    131,
    36,
    66,
    701,
    631,
    13,
    657,
    585,
    1502,
    207,
    235,
    138,
    465,
    91,
    201,
    14,
    14,
    14,
    3,
    3,
    8,
    3,
    3,
    3,
    3,
    7,
    8,
    3,
    3,
    3,
    230,
    3,
    5,
    27,
    2,
    2,
    4,
    4,
    4,
    4,
    3,
    2,
    2,
    2,
    2,
    5,
    2,
    88,
    2,
    2,
    271,
    200,
    5,
    5,
    5,
    3,
    5,
    5,
    1,
    1,
    1,
    1,
    51,
    1,
    1,
    1,
    2,
    147,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    6,
    3,
    2,
    2,
    2,
    6,
    6,
    6,
    38,
    6,
    6,
    8,
    151,
    9,
    434,
    125,
    20,
    6,
    8,
    234,
    29,
    96,
    328,
    114,
    140,
    345,
    5,
    2,
    9,
    9,
    9,
    3,
    2,
    3,
    1,
    46,
    1,
    1,
    3,
    2,
    2,
    3,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    204,
    81,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    245,
    37,
    23,
    54,
    18,
    18,
    18,
    17,
    68,
    69,
    49,
    49,
    36,
    18,
    18,
    6,
    6,
    135,
    92,
    7,
    11,
    11,
    11,
    6,
    6,
    3,
    146,
    2,
    2,
    2,
    3,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    24,
    88,
    543,
    76,
    48,
    11,
    3,
    1,
    1,
    1,
    3,
    3,
    7,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    2,
    2,
    1,
    1,
    19,
    780,
    147,
    518,
    365,
    3,
    3,
    3,
    10,
    2,
    2,
    78,
    2,
    2,
    2,
    2,
    3,
    1,
    2,
    2,
    1,
    1,
    20,
    2,
    2,
    1,
    1,
    85,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    4,
    4,
    10,
    3,
    10,
    10,
    10,
    10,
    15,
    15,
    14,
    3,
    3,
    182,
    165,
    46,
    4,
    4,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    35,
    3,
    4,
    2,
    31,
    3,
    70,
    3,
    4,
    4,
    3,
    3,
    3,
    2,
    34,
    2,
    3,
    346,
    39,
    5,
    6,
    8,
    9,
    6,
    44,
    7,
    8,
    2,
    2,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    2,
    2,
    2,
    2,
    83,
    3,
    3,
    3,
    8,
    9,
    9,
    2,
    2,
    2,
    3,
    13,
    8,
    524,
    41,
    1263,
    48,
    41,
    47,
    36,
    49,
    18,
    18,
    18,
    47,
    14,
    34,
    661,
    128,
    118,
    149,
    36,
    37,
    3,
    3,
    50,
    54,
    50,
    118,
    119,
    136,
    118,
    118,
    118,
    118,
    85,
    85,
    577,
    23,
    22,
    1684,
    1426,
    469,
    2225,
    554,
    50,
    1,
    60,
    61,
    68,
    39,
    68,
    17,
    17,
    135,
    2,
    2,
    2,
    17,
    144,
    15,
    1,
    5,
    331,
    6,
    3,
    12,
    13,
    7,
    6,
    6,
    6,
    6,
    1,
    1,
    15,
    15,
    14,
    5,
    3,
    82,
    17,
    17,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    52,
    52,
    51,
    674,
    346,
    140,
    159,
    214,
    21,
    221,
    221,
    221,
    1,
    140,
    140,
    102,
    195,
    221,
    21,
    21,
    809,
    159,
    661,
    301,
    21,
    966,
    888,
    966,
    297,
    297,
    297,
    297,
    22,
    17,
    17,
    16,
    14,
    14,
    7,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    5,
    3,
    2529,
    761,
    17,
    38,
    82,
    1,
    24,
    16,
    21,
    38,
    38,
    38,
    38,
    37,
    6110,
    6086,
    6687,
    3271,
    226,
    294,
    3033,
    3183,
    3396,
    1251,
    889,
    1832,
    2300,
    2238,
    1157,
    1149,
    1149,
    3198,
    3326,
    2838,
    2889,
    1850,
    1542,
    604,
    869,
    2305,
    2030,
    1119,
    852,
    3388,
    2161,
    1431,
    3487,
    2291,
    3721,
    3768,
    2844,
    1248,
    552,
    3768,
    1809,
    2088,
    2024,
    499,
    1577,
    48,
    7262,
    3616,
    3919,
    1720,
    2493,
    437,
    840,
    1318,
    2576,
    445,
    810,
    1967,
    2093,
    2043,
    1838,
    1217,
    1363,
    1420,
    1672,
    3674,
    2586,
    2268,
    1039,
    339,
    380,
    1277,
    1351,
    901,
    251,
    1089,
    2116,
    946,
    3412,
    2265,
    232,
    1613,
    232,
    250,
    912,
    1478,
    1336,
    1300,
    1234,
    1362,
    936,
    657,
    6017,
    2378,
    3610,
    637,
    1008,
    1314,
    1876,
    2239,
    1001,
    815,
    3,
    237,
    562,
    846,
    701,
    399,
    1397,
    3090,
    1861,
    581,
    2963,
    1917,
    2107,
    2429,
    997,
    1595,
    3067,
    306,
    2344,
    69,
    21,
    31,
    30,
    5,
    6,
    9,
    966,
    1187,
    436,
    1115,
    1192,
    118,
    7,
    6,
    6,
    24,
    3,
    3,
    5,
    21,
    30,
    30,
    107,
    8,
    163,
    22,
    8,
    912,
    66,
    3,
    3,
    3,
    5,
    1,
    26,
    4,
    4,
    4,
    2,
    6,
    6,
    2,
    82,
    3,
    3,
    3,
    2,
    2,
    2,
    2,
    2,
    110,
    2,
    2,
    2,
    7,
    85,
    4,
    3,
    3,
    3,
    3,
    2,
    1,
    3,
    3,
    3,
    1,
    1,
    72,
    1004,
    798,
    1312,
    446,
    580,
    681,
    1265,
    148,
    735,
    474,
    155,
    46,
    17,
    2,
    44,
    37,
    37,
    63,
    68,
    23,
    130,
    1089,
    20,
    131,
    106,
    106,
    717,
    1913,
    1746,
    298,
    301,
    80,
    42,
    29,
    370,
    316,
    354,
    736,
    31,
    31,
    170,
    31,
    2302,
    1066,
    13,
    1625,
    6,
    6,
    1,
    1,
    2,
    2,
    2,
    107,
    2,
    2,
    2,
    3,
    6,
    3,
    1,
    1,
    1,
    154,
    849,
    4,
    5,
    4,
    464,
    2,
    7,
    2,
    2,
    2,
    2,
    425,
    2,
    2,
    2,
    2,
    756,
    444,
    59,
    221,
    1,
    1,
    146,
    26,
    6,
    249,
    353,
    8,
    6,
    630,
    21,
    3,
    351,
    14,
    8,
    8,
    8,
    7,
    4,
    4,
    9,
    50,
    50,
    50,
    165,
    2,
    107,
    2,
    2,
    794,
    469,
    1468,
    845,
    151,
    10,
    1417,
    8,
    50,
    147,
    259,
    4,
    46,
    4,
    4,
    6,
    6,
    6,
    6,
    147,
    7,
    32,
    1016,
    871,
    1248,
    8,
    8,
    4,
    200,
    7,
    3,
    30,
    491,
    539,
    21,
    8,
    51,
    46,
    7,
    7,
    7,
    1,
    1,
    1830,
    1830,
    1366,
    2504,
    1943,
    2323,
    1106,
    2212,
    1569,
    1410,
    1726,
    1962,
    1733,
    1016,
    1016,
    2040,
    1818,
    1818,
    2411,
    2331,
    2051,
    2638,
    1951,
    2404,
    2080,
    2480,
    2496,
    2561,
    2618,
    2513,
    2538,
    2210,
    1625,
    2480,
    2812,
    2860,
    2770,
    1989,
    2629,
    1272,
    890,
    853,
    377,
    2833,
    1446,
    2838,
    1688,
    2524,
    1698,
    2617,
    2270,
    2289,
    2458,
    1906,
    2474,
    2398,
    1869,
    1557,
    2411,
    2944,
    2377,
    1753,
    808,
    1495,
    2725,
    1728,
    945,
    1728,
    1119,
    1861,
    683,
    1726,
    1337,
    1496,
    1117,
    2063,
    860,
    2639,
    1643,
    1344,
    589,
    602,
    1447,
    1762,
    1691,
    573,
    295,
    405,
    191,
    45,
    213,
    931,
    1170,
    1963,
    268,
    85,
    20,
    52,
    52,
    52,
    52,
    47,
    52,
    52,
    52,
    52,
    365,
    157,
    158,
    92,
    157,
    157,
    266,
    21,
    1258,
    1118,
    3,
    1357,
    1568,
    1120,
    1123,
    1573,
    1573,
    826,
    827,
    65,
    36,
    459,
    416,
    18,
    7,
    7,
    7,
    5,
    121,
    72,
    72,
    68,
    1,
    1,
    1,
    1,
    20,
    20,
    20,
    19,
    10,
    2,
    2,
    2,
    48,
    47,
    47,
    47,
    40,
    44,
    436,
    413,
    176,
    183,
    21,
    41,
    5,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    46,
    1,
    1,
    1,
    1,
    2,
    2,
    2,
    22,
    176,
    63,
    1,
    1,
    1,
    1,
    36,
    6,
    200,
    8,
    296,
    8,
    8,
    8,
    34,
    37,
    241,
    8,
    2,
    129,
    2,
    1,
    9,
    54,
    9,
    9,
    1,
    1,
    10,
    2,
    4,
    3,
    3,
    3,
    3,
    3,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    3,
    3,
    47,
    2,
    3,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    1,
    1,
    1,
    1,
    1,
    2,
    2,
    1,
    1,
    65,
    4,
    2,
    2,
    2,
    2,
    2,
    172,
    2,
    8,
    14,
    2,
    32,
    20,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    533,
    1,
    1,
    124,
    40,
    533,
    533,
    1,
    1,
    1,
    5,
    54,
    1,
    56,
    19,
    14,
    1,
    3284,
    3088,
    2361,
    2021,
    1646,
    2140,
    2076,
    2592,
    719,
    3142,
    2362,
    2503,
    3182,
    1909,
    1967,
    1613,
    2163,
    1656,
    1717,
    1581,
    1448,
    2146,
    1452,
    1798,
    662,
    2,
    2,
    486,
    619,
    2354,
    75,
    486,
    1,
    1,
    1,
    1,
    2493,
    2756,
    2161,
    1850,
    1428,
    481,
    671,
    1217,
    778,
    40,
    3139,
    2791,
    2069,
    495,
    2084,
    58,
    806,
    806,
    492,
    46,
    134,
    132,
    116,
    116,
    116,
    292,
    63,
    265,
    116,
    115,
    8,
    17,
    4,
    2205,
    2877,
    1928,
    1456,
    2,
    1675,
    2127,
    297,
    1333,
    543,
    833,
    26,
    708,
    26,
    8,
    11,
    8,
    1464,
    2170,
    1800,
    1767,
    2316,
    2185,
    2303,
    2345,
    2196,
    2427,
    2425,
    2570,
    604,
    604,
    604,
    1768,
    1771,
    1196,
    3,
    948,
    1124,
    943,
    2002,
    1153,
    661,
    1625,
    1625,
    1625,
    718,
    666,
    13,
    67,
    161,
    1131,
    93,
    1,
    13,
    5,
    6,
    6,
    6,
    6,
    6,
    5,
    48,
    7,
    7,
    141,
    739,
    1296,
    281,
    7,
    22,
    4,
    4,
    2,
    2,
    4,
    4,
    4,
    2,
    2,
    2,
    15,
    1,
    1,
    11,
    1,
    1,
    10,
    10,
    11,
    76,
    3,
    3,
    3,
    1,
    1,
    1,
    1,
    1,
    9,
    1,
    2,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    266,
    4,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    8,
    8,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    472,
    575,
    201,
    24,
    1,
    5,
    5,
    1,
    5,
    5,
    1,
    1,
    1,
    1,
    2,
    356,
    80,
    28,
    28,
    8,
    8,
    8,
    3,
    3,
    1,
    61,
    128,
    46,
    524,
    250,
    315,
    167,
    13,
    45,
    1481,
    606,
    1678,
    650,
    1344,
    1912,
    1096,
    330,
    168,
    815,
    101,
    153,
    1164,
    547,
    62,
    86,
    62,
    62,
    62,
    36,
    7,
    3,
    101,
    272,
    505,
    451,
    147,
    225,
    16,
    21,
    15,
    5,
    5,
    3,
    3,
    72,
    32,
    8,
    8,
    8,
    86,
    28,
    28,
    28,
    28,
    28,
    28,
    27,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    2,
    1,
    1,
    1,
    19,
    1,
    2,
    2,
    2,
    520,
    1023,
    667,
    7,
    10,
    6,
    1,
    2,
    1,
    218,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    4,
    4,
    16,
    18,
    18,
    110,
    21,
    21,
    8,
    25,
    19,
    2,
    7,
    3,
    6,
    6,
    22,
    839,
    204,
    74,
    582,
    21,
    155,
    80,
    26,
    1,
    1,
    1,
    1,
    1,
    1,
    6733,
]

f2 = [
    0.000378296,
    0.001210416,
    0.000654749,
    4.87285e-07,
    0.00109677,
    0.000350195,
    0.000204551,
    8.47876e-05,
    0.000661462,
    0.00062719,
    8.16473e-05,
    0.000159884,
    0.000366384,
    2.64217e-05,
    4.38556e-05,
    1.91124e-05,
    3.28647e-05,
    0.000340883,
    2.92912e-05,
    9.08516e-05,
    0.000871428,
    7.55833e-05,
    1.68384e-05,
    7.10895e-05,
    0.000131188,
    3.36227e-05,
    0.001121351,
    0.000577,
    0.000504936,
    0.000503041,
    0.000576675,
    0.000419769,
    0.000357938,
    0.000322799,
    0.000222798,
    0.000870237,
    0.000266924,
    0.000315436,
    0.000514573,
    0.000908407,
    0.000339529,
    0.000273746,
    0.001034344,
    0.001041436,
    0.00040136,
    0.000266653,
    0.0002921,
    0.00070705,
    0.00047689,
    0.000401036,
    0.000202115,
    0.000288364,
    0.000240665,
    0.000978956,
    0.000396704,
    0.000201898,
    0.000323936,
    0.000223122,
    1.02871e-06,
    0.000570827,
    0.000173852,
    0.000222635,
    0.000460214,
    0.001090165,
    0.001096933,
    0.000818422,
    0.000501146,
    0.000469255,
    0.000462054,
    0.000283925,
    0.000693244,
    0.000812791,
    0.0007652,
    0.000839105,
    0.001251564,
    0.000274991,
    0.000111101,
    0.00019264,
    0.000196105,
    6.70829e-05,
    0.000178238,
    0.00051571,
    0.000449439,
    0.000351495,
    0.000408345,
    0.000249923,
    0.00076742,
    0.000190528,
    0.000499954,
    0.000500875,
    0.001431102,
    0.000934992,
    0.000731361,
    0.00045415,
    0.000159017,
    8.23512e-05,
    0.000391344,
    0.000624212,
    7.99147e-05,
    2.00328e-06,
    0.00034673,
    0.000571044,
    0.001181883,
    0.001063851,
    0.000755346,
    0.000397679,
    0.000409969,
    0.000708296,
    0.000879766,
    0.000411214,
    0.0007639,
    0.000928711,
    0.000626486,
    0.000636773,
    1.61345e-05,
    0.000195943,
    0.000555992,
    0.000330812,
    0.000348842,
    0.000755671,
    0.000285711,
    0.00101247,
    0.000636502,
    0.001036293,
    0.000910952,
    0.000350953,
    0.001171975,
    0.002645416,
    0.000595354,
    0.0014167,
    0.002378221,
    0.00122406,
    0.000734934,
    0.000393889,
    0.002297711,
    0.001565971,
    0.00060001,
    0.000181649,
    0.000391019,
    0.000230323,
    0.001175765,
    0.001088595,
    0.000620368,
    0.000340775,
    0.001128444,
    0.001302783,
    0.001674636,
    0.001380208,
    0.004118154,
    0.001399591,
    9.16096e-05,
    0.000455395,
    7.76407e-05,
    0.001682649,
    0.00055074,
    0.000157231,
    0.000453608,
    0.001075438,
    0.000205255,
    0.00047088,
    0.000533685,
    0.000664169,
    0.001069212,
    0.000318414,
    0.000315977,
    0.000613004,
    0.001653737,
    0.000376563,
    0.000769369,
    0.00066287,
    0.000901802,
    0.000485282,
    0.000519662,
    0.001131476,
    0.000638722,
    0.000624483,
    0.00078258,
    0.000197621,
    0.000829251,
    0.000836127,
    0.00045821,
    0.000490425,
    0.000857188,
    0.000598332,
    0.000352848,
    0.000700391,
    0.000566983,
    0.000200924,
    0.000427132,
    0.000530058,
    0.000706347,
    0.001249453,
    0.001134941,
    0.000106607,
    0.001330396,
    0.001090273,
    0.001209441,
    0.002950565,
    0.001340088,
    0.000566225,
    0.001133587,
    0.00023525,
    0.002668535,
    0.001030878,
    0.002149522,
    0.00081696,
    0.000935533,
    0.000680575,
    0.000293887,
    0.000126369,
    0.000117111,
    0.001089136,
    0.000623671,
    0.000686585,
    0.001019184,
    0.000513057,
    0.00138703,
    0.000190258,
    0.001492554,
    0.000736937,
    0.001385568,
    0.000537421,
    0.000162428,
    0.000162049,
    9.42084e-05,
    0.000290043,
    9.0635e-05,
    7.50419e-05,
    0.000347759,
    0.000441264,
    0.000562706,
    7.44463e-05,
    0.0003875,
    0.000687018,
    0.000529841,
    9.44791e-05,
    0.000159342,
    1.01788e-05,
    4.40722e-05,
    7.53126e-05,
    1.29943e-06,
    0.000647169,
    0.00028896,
    0.000285874,
    0.000559674,
    0.000285549,
    0.001609935,
    0.000770181,
    0.001108573,
    0.001050857,
    0.001065638,
    0.001177497,
    0.000324315,
    0.000337526,
    0.000104387,
    9.83774e-05,
    0.001036239,
    5.95571e-07,
    0.000597682,
    0.000128156,
    0.000333519,
    0.000109639,
    0.000352686,
    6.221e-05,
    3.88204e-05,
    0.002151255,
    6.28598e-05,
    2.274e-05,
    0.000227616,
    3.34602e-05,
    3.37309e-05,
    0.000135628,
    8.58704e-05,
    0.000991192,
    2.54471e-06,
    0.000119926,
    5.02445e-05,
    0.000174827,
    3.15111e-05,
    0.000178888,
    0.000175747,
    3.57884e-05,
    0.00059909,
    3.98491e-05,
    0.000160912,
    3.86038e-05,
    0.000329026,
    5.59295e-05,
    0.000212348,
    1.78671e-05,
    0.000468443,
    5.94488e-05,
    6.80033e-05,
    0.000382356,
    3.58967e-05,
    4.14734e-05,
    0.000390369,
    1.34816e-05,
    7.64496e-05,
    3.15111e-05,
    8.40837e-05,
    8.39213e-06,
    1.95997e-05,
    0.000192857,
    9.33421e-05,
    0.000260589,
    0.000169359,
    2.33355e-05,
    9.44791e-05,
    3.44348e-05,
    0.000158584,
    0.001073597,
    0.000448573,
    0.001552923,
    0.000237578,
    0.000164594,
    0.000953671,
    0.000240448,
    0.000396271,
    0.002081736,
    0.000307748,
    0.00063948,
    0.00033888,
    0.000378566,
    9.65907e-05,
    0.001957911,
    0.000574455,
    0.000133245,
    0.000337743,
    0.000139309,
    0.000682145,
    0.001277282,
    0.000118681,
    0.00034267,
    0.000277807,
    1.57555e-05,
    7.41215e-05,
    0.000189825,
    0.000612896,
    0.000111913,
    9.31797e-05,
    0.000125557,
    0.000280026,
    1.49975e-05,
    2.71255e-05,
    0.000535418,
    0.001310688,
    0.000253984,
    0.001989855,
    0.000616903,
    0.000335848,
    0.000487989,
    0.000819018,
    0.000475969,
    0.001173707,
    0.000538071,
    0.000119222,
    0.00026844,
    2.66924e-05,
    0.001540524,
    0.000408507,
    0.000219387,
    0.000276615,
    0.000364273,
    0.000434117,
    0.000250031,
    0.000494973,
    0.000708945,
    0.000641321,
    3.24857e-07,
    0.00044592,
    0.000736233,
    0.00103077,
    0.000902289,
    0.001166831,
    0.001324711,
    0.000730873,
    0.000248569,
    0.000361024,
    0.000852316,
    0.000608186,
    0.000122092,
    0.000914905,
    0.002094188,
    0.001180908,
    5.90698e-05,
    0.000446894,
    0.000352036,
    0.000194102,
    0.000806402,
    0.000160425,
    6.82199e-06,
    0.000118031,
    0.000632983,
    0.000393943,
    7.17392e-05,
    0.002762743,
    9.96227e-06,
    0.000974787,
    5.44135e-05,
    6.65415e-05,
    0.00231547,
    0.000575213,
    0.000213106,
    0.00021917,
    0.000267682,
    0.000800988,
    0.000628868,
    0.000790322,
    0.00012442,
    0.000233518,
    0.001424442,
    0.000122309,
    0.000613708,
    0.000249652,
    2.86957e-05,
    1.60263e-05,
    0.000142341,
    0.000141529,
    1.90041e-05,
    0.000317223,
    0.000576025,
    0.000347976,
    0.000127831,
    0.001054972,
    0.000363298,
    0.000409807,
    0.000424533,
    0.00052643,
    0.000557887,
    0.000423397,
    0.001227904,
    0.000485607,
    0.000716525,
    0.001732623,
    0.00126261,
    6.8653e-05,
    0.000380624,
    0.001875127,
    0.000225126,
    5.14356e-06,
    1.35898e-05,
    2.36604e-05,
    0.000232218,
    0.000108069,
    2.52847e-05,
    0.000432276,
    3.39475e-05,
    9.74029e-05,
    1.79213e-05,
    0.001531699,
    0.000612625,
    0.000237578,
    0.0001222,
    0.000118789,
    2.05201e-05,
    0.000303687,
    8.98229e-05,
    0.000103304,
    1.57014e-06,
    0.002490622,
    8.87942e-05,
    0.000242993,
    0.000116624,
    6.67039e-05,
    5.78245e-05,
    6.56752e-05,
    4.32059e-05,
    7.0927e-05,
    6.06399e-06,
    0.000183165,
    0.000103304,
    0.000249869,
    2.17113e-05,
    8.36506e-05,
    0.000183057,
    6.81658e-05,
    1.19114e-06,
    0.001088432,
    9.30714e-05,
    0.000220145,
    0.000105687,
    3.79541e-05,
    3.51387e-05,
    0.000469743,
    5.39262e-05,
    6.69205e-05,
    0.000440993,
    6.17769e-05,
    0.000116894,
    0.000113808,
    5.27351e-05,
    4.34766e-05,
    0.00038214,
    1.20738e-05,
    0.000143695,
    2.00328e-05,
    1.49434e-05,
    2.09533e-05,
    8.31633e-05,
    7.16309e-05,
    0.000222527,
    6.80575e-05,
    4.61838e-05,
    0.001161958,
    0.00172683,
    0.000102492,
    0.001910807,
    0.000256149,
    0.000723726,
    0.000286524,
    9.20427e-07,
    0.001564889,
    4.26645e-05,
    0.000436228,
    0.001012578,
    0.001220486,
    0.001168239,
    0.000540399,
    0.001018155,
    0.001361799,
    0.000574563,
    0.00072443,
    0.000877221,
    0.001861374,
    0.000235846,
    0.000495136,
    7.98606e-05,
    0.000162158,
    0.000703098,
    0.000100002,
    0.000188092,
    0.000502662,
    0.000246079,
    0.00011148,
    5.06235e-05,
    0.000190907,
    5.37638e-05,
    0.000230215,
    0.000680196,
    1.7055e-05,
    1.92748e-05,
    5.80952e-05,
    0.000461296,
    0.001908966,
    2.32814e-05,
    0.000105416,
    0.000198217,
    0.002008264,
    1.84627e-05,
    0.000128102,
    2.5014e-05,
    4.23397e-05,
    2.28483e-05,
    1.06661e-05,
    1.39688e-05,
    0.00014716,
    7.52585e-06,
    3.15111e-05,
    0.000279647,
    0.000202765,
    1.4023e-05,
    6.54045e-05,
    0.000158097,
    0.001850221,
    0.000127181,
    3.12404e-05,
    5.81493e-05,
    6.45382e-05,
    8.84152e-05,
    1.83003e-05,
    0.0001137,
    7.2822e-05,
    8.85234e-05,
    0.000893518,
    0.000174394,
    4.29352e-05,
    4.32059e-05,
    0.000140176,
    2.71255e-05,
    3.16194e-05,
    6.96818e-05,
    0.000518471,
    0.000413867,
    4.19607e-05,
    2.38228e-05,
    0.000710461,
    1.12076e-05,
    5.41428e-07,
    0.000131729,
    0.000613275,
    0.000365734,
    0.000525618,
    0.001142629,
    0.00070613,
    0.000227508,
    0.000243209,
    0.001365264,
    0.001071865,
    0.00163365,
    0.000289393,
    0.000637639,
    0.000387283,
    0,
    0.000645111,
    0.001006622,
    0.000503311,
    0.000632171,
    0.00062719,
    0.000534931,
    0.000405042,
    0.001701978,
    0.001855798,
    0.000147647,
    0.002362466,
    0.000988647,
    0.000493511,
    0.000431301,
    0.000290801,
    0.001036184,
    0.004219184,
    0.004204457,
    0.00052421,
    0.000954808,
    0.00025772,
    0.001876968,
    0.001099369,
    0.000527296,
    0.000954375,
    0.000581656,
    0.000741864,
    0.001437328,
    0.000959085,
    1.52141e-05,
    0.00121648,
    0.000698009,
    0.000628814,
    0.000389395,
    9.96227e-06,
    0.00014781,
    0.000323124,
    0.000219603,
    0.000419769,
    0.000364543,
    0.001195418,
    0.000790593,
    0.00049833,
    0.000504178,
    3.032e-05,
    0.00352816,
    0.000289339,
    0.000308181,
    0.000268656,
    0.000539045,
    0.000749336,
    0.000598007,
    0.001311717,
    0.000942788,
    0.00080088,
    0.001261527,
    0.000457073,
    0.000616361,
    0.000532115,
    0.00114734,
    0.001174032,
    0.000972675,
    0.000484578,
    1.62428e-07,
    0.000325019,
    0.001057625,
    0.000504557,
    0.000116461,
    8.37047e-05,
    0.000166272,
    0.000350033,
    0.001174303,
    7.92109e-05,
    7.35259e-05,
    0.000227562,
    0.000904996,
    0.000243967,
    0.000577379,
    0.000760923,
    0.000439369,
    0.000679113,
    0.000518525,
    0.000986481,
    0.001095146,
    0.000499467,
    0.000369091,
    0.000176993,
    0.000181108,
    0.000460593,
    0.00074717,
    0.000512624,
    0.000805699,
    0.000562706,
    0.000304986,
    0.000678842,
    0.000705805,
    0.000594921,
    0.001242956,
    0.000889295,
    0.00063921,
    0.000874298,
    0.001407766,
    0.000736125,
    0.001800843,
    0.000633525,
    0.000851124,
    0.000670071,
    0.000777003,
    0.000693136,
    0.000539695,
    0.000318143,
    0.000458752,
    0.000773646,
    0.000879116,
    0.000506397,
    0.000345485,
    0.001361853,
    0.002287316,
    0.00042475,
    0.001343174,
    0.001042411,
    0.000181053,
    0.00089563,
    0.001620602,
    0.000277536,
    0.00011305,
    0.000370824,
    0.000514194,
    0.00066601,
    0.001033152,
    0.001044956,
    0.000907216,
    0.00040266,
    0.000758811,
    0.001044198,
    0.000809759,
    0.000888483,
    0.001170404,
    0.00138952,
    0.002565122,
    0.000412514,
    0.001444204,
    0.000281001,
    0.000375588,
    0.00056655,
    0.000683065,
    0.000501037,
    0.000510025,
    0.000467794,
    0.000936074,
    0.000616578,
    0.000659134,
    0.000422368,
    0.000420527,
    0.000404663,
    0.00099195,
    0.000807702,
    0.000538287,
    0.001539063,
    0.000957623,
    0.000485715,
    0.000745384,
    0.002180763,
    0.000180999,
    0.00098973,
    0.000120738,
    0.001125466,
    0.000349492,
    0.000989784,
    7.31469e-05,
    0.000745979,
    0.000147539,
    0.000930281,
    0.002315795,
    0.000605966,
    0.000553447,
    0.000803912,
    0.000986481,
    0.000256312,
    0.000778898,
    0.000473533,
    0.000605479,
    0.000261347,
    0.000985128,
    0.000165135,
    8.96063e-05,
    8.08893e-05,
    0.000114729,
    1.12617e-05,
    2.45267e-05,
    1.33191e-05,
    1.44561e-05,
    1.55931e-05,
    2.78835e-05,
    0.000114133,
    0.000151708,
    5.09484e-05,
    2.29024e-05,
    0.000174177,
    0.000572993,
    3.16194e-05,
    1.5539e-05,
    0.000240177,
    0.000120576,
    0.000141367,
    0.000156581,
    1.4402e-05,
    0.000145427,
    3.47597e-05,
    1.46727e-05,
    1.23987e-05,
    8.116e-05,
    7.90485e-06,
    1.64594e-05,
    2.18195e-05,
    2.54471e-05,
    9.09599e-05,
    1.86793e-05,
    1.59721e-05,
    0.001464616,
    8.93356e-05,
    0.000126099,
    6.53503e-05,
    0.000179429,
    6.6e-05,
    3.24315e-05,
    2.45808e-05,
    4.34766e-05,
    9.58327e-06,
    2.20903e-05,
    5.53339e-05,
    0.000491887,
    6.61083e-05,
    2.08991e-05,
    1.84627e-05,
    0.000147214,
    0.000647818,
    6.60542e-06,
    1.67301e-05,
    9.17179e-05,
    5.35472e-05,
    3.1078e-05,
    0.000100706,
    1.1749e-05,
    7.21723e-05,
    2.49598e-05,
    8.33257e-05,
    1.14783e-05,
    6.28056e-06,
    3.34061e-05,
    3.25398e-05,
    9.35046e-05,
    1.68925e-05,
    1.22363e-05,
    0.000309372,
    5.67416e-05,
    4.65628e-05,
    3.21608e-05,
    0.000228916,
    0.000427024,
    0.001580211,
    0.001094117,
    8.97146e-05,
    0.000194968,
    6.62708e-05,
    0.002653646,
    0.000459401,
    0.000917124,
    0.000660488,
    0.00066731,
    0.000167247,
    0.000100706,
    4.43429e-05,
    3.97949e-05,
    0.000169034,
    0.000123662,
    0.000100381,
    6.45923e-05,
    0.000141962,
    2.24151e-05,
    1.32108e-05,
    0.000421339,
    9.58327e-06,
    4.21231e-05,
    0.000302387,
    6.54045e-05,
    3.66547e-05,
    3.21608e-05,
    2.30107e-05,
    3.34602e-05,
    4.85119e-05,
    9.63741e-06,
    3.2215e-05,
    0.000143695,
    2.78835e-05,
    3.83331e-05,
    0.000683986,
    0.0002457,
    2.60968e-05,
    5.62002e-05,
    2.03577e-05,
    1.15324e-05,
    9.07974e-05,
    2.49057e-06,
    0.00015317,
    5.24643e-05,
    1.53765e-05,
    2.48515e-05,
    4.75374e-05,
    3.49762e-05,
    0.001570628,
    1.87334e-05,
    0.00020688,
    9.85399e-06,
    1.18031e-05,
    0.000289393,
    0.000163836,
    0.000507318,
    0.000113483,
    9.73487e-05,
    6.96276e-05,
    0.000377159,
    0.00048317,
    5.45218e-05,
    2.81542e-05,
    5.47925e-05,
    0.000316519,
    0.000267032,
    9.43709e-05,
    0.000125124,
    0.000144182,
    0.000247757,
    6.69746e-05,
    0.000149217,
    6.07482e-05,
    0.000480625,
    7.64496e-05,
    1.80837e-05,
    1.60804e-05,
    8.6087e-05,
    2.30107e-05,
    4.72666e-05,
    0.00013801,
    9.81067e-05,
    1.10451e-05,
    5.32223e-05,
    1.895e-05,
    4.49385e-06,
    8.28384e-06,
    3.95242e-06,
    8.77113e-06,
    0.000318684,
    0.000275316,
    0.001516323,
    0.000149813,
    0.000243318,
    0.00010769,
    4.13109e-05,
    7.83446e-05,
    8.31633e-05,
    9.3667e-06,
    3.90369e-05,
    1.87875e-05,
    0.000225288,
    7.21723e-05,
    4.17982e-05,
    7.95899e-06,
    7.74242e-06,
    1.35898e-05,
    4.36391e-05,
    5.37096e-05,
    1.80295e-05,
    1.10451e-05,
    1.65677e-05,
    2.60968e-05,
    0.000225234,
    0.001800951,
    0.001557092,
    0.000868071,
    0.000604721,
    4.97572e-05,
    5.45218e-05,
    3.02117e-05,
    0.000243047,
    2.00328e-05,
    3.96867e-05,
    0.000895197,
    5.57671e-06,
    3.1836e-05,
    3.97408e-05,
    0.000168546,
    4.23938e-05,
    2.0466e-05,
    1.20197e-05,
    2.0087e-05,
    4.53716e-05,
    4.33142e-06,
    0.000274341,
    6.38343e-05,
    1.2886e-05,
    1.11534e-05,
    5.72831e-05,
    0.000562435,
    3.73044e-05,
    1.84627e-05,
    2.27941e-05,
    1.75423e-05,
    1.07744e-05,
    3.42724e-05,
    3.45431e-05,
    7.74242e-06,
    3.27022e-05,
    8.55456e-06,
    7.52585e-06,
    0.000149109,
    7.76949e-05,
    0.00010455,
    4.57506e-05,
    0.000282355,
    7.14685e-06,
    3.42182e-05,
    0.000408561,
    0.000111263,
    2.71797e-05,
    4.93782e-05,
    2.653e-05,
    0.000419444,
    0.001455791,
    0.000617823,
    3.29729e-05,
    9.88106e-05,
    3.21067e-05,
    0.000263242,
    2.94537e-05,
    0.000110776,
    3.40017e-05,
    5.46842e-05,
    6.6379e-05,
    0.000477756,
    3.21067e-05,
    4.87285e-07,
    2.38228e-05,
    0.000323557,
    4.39639e-05,
    0.000366763,
    2.93995e-05,
    1.27777e-05,
    5.92322e-05,
    2.54471e-05,
    0.000263729,
    7.83446e-05,
    3.43807e-05,
    0.000508888,
    0.000100489,
    0.000100056,
    0.000828222,
    0.000343428,
    3.62215e-05,
    9.38294e-05,
    0.000160425,
    5.84742e-05,
    7.4338e-05,
    0.001696456,
    0.00056211,
    2.68548e-05,
    3.00492e-05,
    4.57506e-05,
    5.18146e-05,
    7.93192e-05,
    6.36719e-05,
    1.516e-06,
    1.76505e-05,
    9.72404e-05,
    5.25185e-06,
    2.21444e-05,
    3.38392e-05,
    2.32814e-05,
    5.96653e-05,
    0.000199895,
    2.36604e-05,
    0.000184085,
    6.76785e-06,
    3.27022e-05,
    4.72125e-05,
    0.000296702,
    0.000177697,
    3.88745e-05,
    2.33897e-05,
    2.14405e-05,
    3.71419e-05,
    0.000490155,
    3.40558e-05,
    1.26153e-05,
    3.88204e-05,
    0.000236875,
    1.02871e-05,
    1.2886e-05,
    6.19393e-05,
    0.000164161,
    2.54471e-05,
    2.4256e-05,
    0.000241044,
    0.000103467,
    0.000680954,
    1.22363e-05,
    0.000928711,
    0.000175368,
    1.34274e-05,
    0.000631684,
    0.000419444,
    0.000110939,
    5.39262e-05,
    2.02494e-05,
    6.13438e-05,
    7.52585e-06,
    8.51666e-05,
    0.000476294,
    0.000287552,
    5.42511e-05,
    5.13815e-05,
    0.00015474,
    0.000192315,
    0.000490534,
    1.32108e-05,
    2.69631e-05,
    0.000193506,
    0.000367359,
    0.000114783,
    2.92371e-06,
    6.62708e-05,
    4.36932e-05,
    8.24594e-05,
    1.4402e-05,
    6.57835e-05,
    4.82412e-05,
    0.000167464,
    7.61247e-05,
    0.000406287,
    0.000172715,
    0.000274829,
    0.000931743,
    0.000574996,
    0.000398978,
    0.002164087,
    0.001971555,
    0.000144074,
    0.002082331,
    0.000141421,
    0.000744788,
    0.000881228,
    0.000155011,
    5.08942e-05,
    0.000103142,
    0.000248028,
    0.000943492,
    9.96768e-05,
    1.01788e-05,
    2.13323e-05,
    2.77211e-05,
    0.000782634,
    0.000200274,
    0.001413884,
    0.000115216,
    0.000663845,
    0.000290043,
    4.06071e-06,
    7.20099e-05,
    5.27351e-05,
    2.17113e-05,
    0.000159667,
    4.74291e-05,
    0.000381165,
    0.000173365,
    1.28318e-05,
    3.01575e-05,
    9.08516e-05,
    7.08187e-05,
    0.000146673,
    1.77047e-05,
    3.73585e-05,
    0.000966178,
    6.47006e-05,
    0.000217221,
    6.62166e-05,
    0.000103792,
    2.32814e-05,
    1.96538e-05,
    0.000177209,
    1.1749e-05,
    1.83003e-05,
    9.52913e-06,
    6.21018e-05,
    1.29943e-06,
    1.08827e-05,
    2.3498e-05,
    3.96867e-05,
    1.29401e-05,
    2.29565e-05,
    4.48844e-05,
    9.6699e-05,
    0.000397462,
    0.000947336,
    0.000170225,
    0.000700662,
    0.000168871,
    0.000432547,
    5.86366e-05,
    0.000196701,
    0.000138118,
    2.6151e-05,
    8.39213e-06,
    0.00047467,
    4.13651e-05,
    0.00020493,
    0.000162753,
    8.31092e-05,
    2.92912e-05,
    6.17769e-05,
    0.000768882,
    6.05316e-05,
    0.000796224,
    0.000607428,
    8.52749e-05,
    0.000164432,
    0.000707754,
    0.000206013,
    0.000641646,
    0.000148351,
    0.000119114,
    6.20476e-05,
    5.38179e-05,
    1.41854e-05,
    0.000240177,
    5.68499e-05,
    0.000398762,
    1.96538e-05,
    0.000157989,
    1.80295e-05,
    1.43478e-05,
    7.06022e-05,
    0.000141096,
    5.14898e-05,
    9.31256e-06,
    1.33733e-05,
    3.20525e-05,
    7.41756e-06,
    0.000224963,
    0.000588424,
    0.0040593,
    0.001308577,
    2.07908e-05,
    0.000161724,
    0.000242505,
    0.000345052,
    8.3434e-05,
    5.61461e-05,
    8.77113e-06,
    2.47432e-05,
    5.36013e-05,
    0.000525618,
    0.000307477,
    0.001587087,
    0.000575538,
    0.000987889,
    0.000616091,
    0.00110213,
    0.000583984,
    0.000195889,
    0.000690212,
    0.001107599,
    0.000289122,
    0.000593567,
    0.001427149,
    0.000458914,
    0.00088426,
    0.001017505,
    0.000377808,
    0.000269252,
    0,
    0.000625674,
    0.000573534,
    0.000632171,
    0.001925859,
    0.001079715,
    0.00026086,
    0.000318793,
    0.001060495,
    0.001747458,
    0.000520637,
    0.000839213,
    0.000409536,
    0.000478027,
    0.000643703,
    0.000894764,
    0.000354094,
    0.000614791,
    0.000269198,
    0.000841812,
    0.001403435,
    0.000462812,
    9.79984e-06,
    0.000368008,
    0.000964337,
    0.000888266,
    0.00039573,
    0.00112774,
    0.000931039,
    0.001025356,
    0.000369416,
    0.000485931,
    0.000254796,
    0.00059373,
    0.000217871,
    0.00033287,
    0.000347759,
    0.001378421,
    0.000721886,
    4.27186e-05,
    0.000306069,
    0.001529642,
    0.000963741,
    0.000730007,
    0.000958869,
    0.000109152,
    0.001223789,
    0.000360158,
    0.000358317,
    0.000910302,
    0.00171573,
    0.000834827,
    0.000946957,
    0.000486365,
    0.001233751,
    0.000699795,
    0.000368171,
    0.000396704,
    0.000436066,
    0.000906783,
    0.00079238,
    0.000672995,
    0.001733543,
    0.000185385,
    0.000425833,
    8.32716e-05,
    0.000146998,
    0.000952263,
    0.00015945,
    0.000600119,
    0.000769423,
    0.000660758,
    0.000696547,
    0.000737804,
    0.001605387,
    0.000565088,
    0.001032394,
    0.0006319,
    0.000443646,
    0.000385767,
    0.000173528,
    0.001073326,
    0.000781389,
    0.000215759,
    0.002800048,
    0.001000288,
    0.002579308,
    0.00095091,
    0.00271082,
    0.001398562,
    0.00135227,
    0.001782922,
    0.002152446,
    0.001684761,
    0.001948436,
    0.001474362,
    0.001551136,
    0.001234997,
    0,
    0.000734772,
    0.000318576,
    0.000463246,
    0.001453571,
    0.000937699,
    0.000269035,
    0.000214297,
    0.000335144,
    0.000184681,
    0.000894709,
    0.001017072,
    3.58967e-05,
    0.002766263,
    0.001190545,
    0.00013866,
    0.000693352,
    0.002475678,
    0.001819035,
    8.08893e-05,
    0.000161291,
    0.000283816,
    0.000482412,
    0.000135628,
    0.000106553,
    0.000127344,
    0.000259073,
    0.000711328,
    0.00012572,
    0.000539587,
    0.000128914,
    0,
    5.01904e-05,
    7.30386e-05,
    0.004655304,
    0.001374577,
    1.80295e-05,
    5.79869e-05,
    0.000125882,
    0.000251277,
    0.000133895,
    0.000125286,
    6.34012e-05,
    3.17818e-05,
    0.000136169,
    0.000127615,
    3.32978e-05,
    1.33733e-05,
    2.77752e-05,
    0.000859896,
    3.70878e-05,
    8.93356e-06,
    5.67958e-05,
    2.20903e-05,
    1.22363e-05,
    1.87875e-05,
    3.1457e-05,
    2.70172e-05,
    0.000377754,
    2.75587e-05,
    2.82084e-05,
    3.08072e-05,
    0.00036541,
    9.5941e-05,
    0.000247595,
    1.53765e-05,
    4.26645e-05,
    2.57178e-05,
    0.000641267,
    7.48795e-05,
    2.82084e-05,
    3.2215e-05,
    6.4484e-05,
    1.22904e-05,
    1.11534e-05,
    0.000345756,
    0.000401144,
    0.001907017,
    0.000243101,
    0.000337959,
    0.001861862,
    0.000828222,
    0.001057679,
    0.000979659,
    0.000862657,
    0.000751502,
    0.002180492,
    0.000423559,
    0.00018035,
    0.000333411,
    0.000197134,
    0.00021473,
    4.36391e-05,
    0.000250194,
    0.000418632,
    0.000654099,
    0.000237687,
    0.000216842,
    0.002723436,
    0.000442346,
    0.000413597,
    9.85399e-05,
    0.000347759,
    0.001975995,
    0.000814686,
    0.001043873,
    0.000318739,
    0.001558554,
    0.000422693,
    0.000446136,
    0.000469147,
    0.000502662,
    0.000666552,
    0.000441913,
    0.000542456,
    0.000189337,
    8.13224e-05,
    0.002122343,
    4.30976e-05,
    0.000186251,
    0.001257845,
    7.36883e-05,
    0.001164557,
    0.000243372,
    8.08352e-05,
    4.67252e-05,
    5.45759e-05,
    0.000173473,
    0.000110451,
    4.68876e-05,
    0.000668068,
    6.93028e-06,
    1.38064e-05,
    4.36932e-05,
    5.56588e-05,
    0.000343265,
    2.47432e-05,
    9.04726e-05,
    0.000132433,
    9.75653e-05,
    0.001488006,
    0.00273865,
    0.000251385,
    0.000375047,
    3.64381e-05,
    0.003536173,
    0.000301196,
    3.27022e-05,
    8.27843e-05,
    9.09599e-06,
    0.00019957,
    4.62921e-05,
    0.002318231,
    9.58327e-06,
    5.27892e-05,
    1.0233e-05,
    2.26317e-05,
    0.000730169,
    0.001702357,
    0.001305436,
    0.001071648,
    8.66284e-06,
    3.032e-06,
    0.000214568,
    8.68992e-05,
    0.000124853,
    0.000319767,
    0.000657402,
    0.0003875,
    6.23725e-05,
    0.003781602,
    0.000342886,
    0.000369362,
    0.00023904,
    0.000208829,
    0.000193019,
    0.000106228,
    3.23774e-05,
    1.75423e-05,
    3.80082e-05,
    5.80952e-05,
    0.00046709,
    5.95571e-07,
    0.000467739,
    1.92207e-05,
    0.000489505,
    0.000186576,
    0.000510512,
    5.45759e-05,
    5.76079e-05,
    0.000935695,
    0.000501687,
    0.000749065,
    0.001834953,
    0.000795899,
    0.000383493,
    0.00121112,
    0.001289952,
    0.000147701,
    0.000440127,
    0.000758757,
    6.74619e-05,
    0.00052773,
    0.000175477,
    0.000248299,
    4.09861e-05,
    3.17818e-05,
    6.79492e-05,
    2.274e-05,
    0.000459293,
    6.68663e-05,
    0.00034581,
    0.000846306,
    0.000741594,
    0.00289718,
    0.000472666,
    0.00021446,
    0.000312566,
    0.000226967,
    0.000111642,
    5.55505e-05,
    0.000215272,
    0.00059752,
    0.000869587,
    0.000584471,
    0.000213485,
    0.000144182,
    0.00037613,
    0.000103575,
    0.000192044,
    4.30435e-05,
    9.87023e-05,
    0.00014272,
    0.001323195,
    1.62428e-07,
    0.000237578,
    0.000375372,
    0.000529029,
    0.000256528,
    0.000258802,
    0.000520204,
    0.000395188,
    0.000513382,
    0.000545109,
    0.000282463,
    0.000367467,
    0.000184573,
    0.000245267,
    0.000347651,
    0.000130484,
    0.000279485,
    0.00064928,
    0.000645544,
    0.000597466,
    0.000359941,
    0.000521124,
    0.00052643,
    0.000463354,
    0.000442617,
    0.000308018,
    0.00044657,
    0.000624537,
    0.000425129,
    9.04726e-05,
    0.000519716,
    0.000446407,
    0.000681333,
    0.000468064,
    0.000631792,
    0.000335306,
    0.000330108,
    0.000581006,
    0.00042762,
    0.000232381,
    0.00061295,
    0.000276074,
    0.000422909,
    0.000625566,
    0.000533144,
    0.000408507,
    0.000413326,
    0.000500767,
    0.0005195,
    0.000499142,
    0.000350195,
    0.000565359,
    0.000517497,
    0.000603313,
    0.000516414,
    0.000436716,
    0.000399411,
    0.000376617,
    0.000589777,
    0.000313757,
    0.000381544,
    0.00082048,
    0.000634499,
    0.000463191,
    0.000159559,
    0.000287715,
    0.000295728,
    0.000326968,
    0.000687992,
    0.000351495,
    0.00015187,
    0.00080716,
    0.000555505,
    0.000548087,
    0.000473316,
    2.70714e-05,
    0.000677489,
    0.001501812,
    0.000802558,
    0.000376834,
    2.33355e-05,
    0.000650688,
    0.000593621,
    0.001317943,
    0.000434333,
    0.000391615,
    0.000163565,
    0.000825786,
    0.000139959,
    9.23676e-05,
    0.000812466,
    0.000298706,
    0.000315598,
    5.29516e-05,
    2.28483e-05,
    2.03577e-05,
    2.12781e-05,
    2.43642e-05,
    3.25398e-05,
    1.66218e-05,
    1.34274e-05,
    1.88417e-05,
    3.63839e-05,
    2.68548e-05,
    2.26317e-05,
    2.97785e-06,
    1.95455e-05,
    0.000292209,
    0.000324857,
    0.000153711,
    0.000167409,
    4.54258e-05,
    0.000161508,
    0.001007435,
    0.00137972,
    8.63577e-05,
    0.000424804,
    0.001370137,
    0.000556479,
    0.00016389,
    0.000233301,
    7.60706e-05,
    0.000255554,
    0.000223501,
    0.000294104,
    0.00020401,
    0.000517605,
    0.000139743,
    0.000109206,
    4.97031e-05,
    6.88696e-05,
    0.000162158,
    9.19344e-05,
    0.000746575,
    0.000100922,
    6.39968e-05,
    0.000448844,
    1.79754e-05,
    2.26858e-05,
    4.63462e-05,
    0.000193019,
    5.70123e-05,
    2.2361e-05,
    7.29845e-05,
    7.06563e-05,
    0.000134437,
    1.66218e-05,
    2.21985e-05,
    0.000234709,
    2.94537e-05,
    0.000117111,
    7.6233e-05,
    4.82954e-05,
    0.000209803,
    0.000147972,
    0.000165514,
    0.000743434,
    0.000282679,
    0.000848526,
    0.000129943,
    0.000482899,
    7.87777e-05,
    5.73913e-05,
    8.5708e-05,
    2.39852e-05,
    2.95078e-05,
    1.5918e-05,
    7.53667e-05,
    2.2361e-05,
    3.41099e-05,
    0.000105633,
    0.000187442,
    1.37523e-05,
    3.53552e-05,
    0.000394051,
    5.27351e-05,
    7.8507e-06,
    0.000136494,
    7.79115e-05,
    0.000157934,
    2.98327e-05,
    2.36062e-05,
    0.000388366,
    0.001171271,
    0.000470826,
    3.89828e-05,
    1.84085e-05,
    4.52634e-05,
    0.000181541,
    0.000294537,
    7.26596e-05,
    0.000374993,
    6.01526e-05,
    0.000236766,
    4.94324e-05,
    0.000169738,
    2.0087e-05,
    3.14028e-05,
    0.000295836,
    0.000268169,
    0.000201519,
    2.51764e-05,
    0.000401415,
    4.43971e-05,
    2.63134e-05,
    6.07482e-05,
    7.90485e-05,
    0.000110126,
    0.000172986,
    8.18639e-05,
    2.24151e-05,
    4.40181e-05,
    4.05529e-05,
    1.91665e-05,
    1.84085e-05,
    6.01526e-05,
    5.01904e-05,
    9.09599e-06,
    1.28318e-05,
    1.1749e-05,
    8.04562e-05,
    7.0927e-06,
    1.2886e-05,
    5.90156e-06,
    7.30927e-06,
    7.88319e-05,
    5.11108e-05,
    3.76292e-05,
    2.33897e-05,
    0.000262755,
    1.99787e-05,
    2.97785e-06,
    3.87121e-05,
    0.000127885,
    2.8425e-05,
    3.16194e-05,
    7.79656e-06,
    6.17228e-06,
    1.01247e-05,
    3.76292e-05,
    4.62379e-05,
    3.43265e-05,
    2.89122e-05,
    1.80837e-05,
    3.23774e-05,
    9.79984e-06,
    9.84316e-05,
    5.36013e-06,
    3.45431e-05,
    0.000414138,
    8.35964e-05,
    7.90485e-06,
    2.87498e-05,
    8.28384e-06,
    7.39049e-05,
    9.53454e-05,
    0.000636611,
    4.45595e-05,
    1.87334e-05,
    0.000262971,
    3.44348e-05,
    0.000201953,
    0.000249273,
    2.32273e-05,
    3.51928e-05,
    0.000103683,
    1.07744e-05,
    0.000118898,
    0.000132812,
    5.90156e-05,
    3.18901e-05,
    2.09533e-05,
    3.13487e-05,
    3.27022e-05,
    4.69418e-05,
    0.000301738,
    1.20197e-05,
    9.77819e-05,
    0.00065058,
    0.000225559,
    0.000472396,
    8.116e-05,
    6.7137e-06,
    5.57671e-05,
    1.41854e-05,
    0.000172391,
    0.000437528,
    9.58327e-06,
    0.000103683,
    8.2676e-05,
    0.000181541,
    0,
    0.000341587,
    0.000357396,
    0.000676514,
    0.000698767,
    0.000713169,
    0.000424967,
    0.000352415,
    0.000820371,
    0.000304012,
    0.0003777,
    0.000742568,
    0.001583514,
    0.000930823,
    0.000956107,
    0.001063797,
    0.000215813,
    0.000633849,
    0.000559187,
    0.001305653,
    0.000942788,
    0.001808531,
    0.00095497,
    0.000128318,
    0.001567487,
    0.000674619,
    1.15866e-05,
    3.05365e-05,
    0.000117111,
    0.000392752,
    0.000587612,
    0.00015788,
    0.000322041,
    8.93897e-05,
    4.65628e-06,
    1.69467e-05,
    7.72617e-05,
    0.000587287,
    0.00054852,
    0.000426807,
    0.000659459,
    0.000973487,
    0.001221299,
    0.001882923,
    0.000734447,
    0.000577216,
    0.000962767,
    0.000925787,
    0.001443176,
    0.001391578,
    0.00057034,
    0.001183128,
    0.003080128,
    0.000751014,
    5.41428e-07,
    3.95242e-06,
    0.001663104,
    0.000170875,
    0.000172661,
    4.34225e-05,
    0.000164215,
    5.03528e-05,
    0.001060116,
    0.000105145,
    5.07318e-05,
    0.000245971,
    0.000164702,
    0.00021094,
    0.000446245,
    0.000255121,
    0.000783825,
    0.000639101,
    0.000578895,
    0.000918478,
    0.000482033,
    0.001899545,
    0.000910248,
    0.000665523,
    0.001670467,
    0.00104344,
    0.002572811,
    0.000175098,
    0.000854427,
    0.000106228,
    6.09648e-05,
    0.000179862,
    1.4023e-05,
    0.000584796,
    0.000717338,
    0.000761626,
    0.000517822,
    0.000739265,
    0.000949935,
    0.000898662,
    0.000478027,
    0.002202149,
    0.000769261,
    0.000562327,
    0.000308993,
    0.000169142,
    0.00039443,
    1.35357e-06,
    0.000471367,
    0.002388292,
    0.002721595,
    4.75915e-05,
    0.000521395,
    0.001064285,
    0.00104972,
    0.00141762,
    0.000579544,
    0.000530383,
    0.000470717,
    0.000204985,
    0.00015788,
    0.000242451,
    2.72338e-05,
    6.16686e-05,
    0.000593242,
    0.000106445,
    0.003145479,
    0.000369958,
    3.78999e-05,
    4.03905e-05,
    0.000231081,
    2.58261e-05,
    0.00024727,
    1.51058e-05,
    1.23987e-05,
    6.70829e-05,
    0.000457723,
    4.78081e-05,
    3.95784e-05,
    1.06661e-05,
    4.20148e-05,
    0.001979135,
    0.000453175,
    0.000883718,
    0.000118681,
    0.000137198,
    0.000243913,
    8.77113e-06,
    4.11485e-06,
    9.37753e-05,
    2.22527e-05,
    6.60542e-06,
    3.78999e-05,
    3.20525e-05,
    0.000218629,
    1.2128e-05,
    0.000328268,
    7.5096e-05,
    4.39639e-05,
    0.000175531,
    8.12683e-05,
    0.001383348,
    0.000394538,
    8.70616e-05,
    0.000133462,
    0.000491833,
    4.30435e-05,
    3.62757e-05,
    0.00010076,
    0.000201086,
    0.000103034,
    5.79328e-05,
    2.05743e-05,
    3.55177e-05,
    0.000211861,
    1.22904e-05,
    2.54471e-05,
    0.000192748,
    2.94537e-05,
    3.54094e-05,
    3.19442e-05,
    1.66218e-05,
    5.92322e-05,
    0.000192694,
    3.25398e-05,
    1.50517e-05,
    3.04824e-05,
    0.001778536,
    0.00031078,
    6.49713e-07,
    4.20689e-05,
    3.08614e-06,
    6.08565e-05,
    5.86908e-05,
    7.57999e-07,
    4.87285e-07,
    6.3347e-06,
    8.82527e-05,
    6.24808e-05,
    3.04824e-05,
    7.74242e-06,
    1.42395e-05,
    0.000113483,
    2.1224e-05,
    5.19771e-06,
    2.78835e-05,
    0.000352253,
    1.40771e-05,
    1.87334e-05,
    5.04069e-05,
    2.5772e-05,
    5.34931e-05,
    8.17556e-06,
    2.38228e-06,
    3.41099e-06,
    2.43642e-06,
    0.000137848,
    4.07154e-05,
    3.68171e-06,
    7.68827e-06,
    2.49057e-06,
    0.000997797,
    0.000326914,
    1.19114e-06,
    0.000661516,
    1.67843e-06,
    5.85825e-05,
    7.38507e-05,
    0.000126694,
    7.0927e-06,
    9.58327e-06,
    3.93618e-05,
    3.05907e-05,
    0.000102384,
    2.55012e-05,
    0.00018414,
    0.001476798,
    0.000326156,
    0.000114025,
    4.19607e-05,
    6.82199e-05,
    7.06022e-05,
    2.40394e-05,
    0.000201953,
    1.35898e-05,
    1.31026e-05,
    0.00022296,
    0.000679005,
    0.000211807,
    0.000417224,
    0.000469959,
    0.000169629,
    0.000171416,
    4.06071e-06,
    0,
    0.001056542,
    0.000290368,
    0.000539858,
    0.000135682,
    0.000527242,
    0.000649172,
    0.000728545,
    0.000620151,
    4.40181e-05,
    0.000456207,
    9.84316e-05,
    0.000300005,
    0.000949285,
    0.000265029,
    0.000134058,
    0.000116136,
    5.06235e-05,
    2.0087e-05,
    2.51764e-05,
    9.5562e-05,
    5.43593e-05,
    1.87334e-05,
    0.000882906,
    0.000575429,
    0.000326752,
    0.000151329,
    0.000426645,
    2.95078e-05,
    0.000479164,
    0.0001699,
    6.11813e-05,
    0.000122038,
    4.94865e-05,
    2.16571e-06,
    0.000112346,
    0.000579274,
    0.000174069,
    4.89992e-05,
    3.22691e-05,
    1.95455e-05,
    0.000380136,
    0.000164811,
    0.000205634,
    0.000366817,
    1.01788e-05,
    1.12617e-05,
    9.69156e-06,
    2.653e-05,
    1.79754e-05,
    2.54471e-05,
    1.05578e-05,
    1.48351e-05,
    2.86957e-06,
    4.73208e-05,
    6.82199e-06,
    1.16948e-05,
    8.6087e-06,
    0.000193939,
    2.68007e-05,
    5.36555e-05,
    1.38064e-05,
    0.000144886,
    1.56473e-05,
    3.89287e-05,
    0.000208666,
    0.000186089,
    0.000667797,
    0.000601743,
    0.002306536,
    0.000161724,
    0.00023498,
    3.41641e-05,
    5.98278e-05,
    8.53832e-05,
    1.73257e-06,
    5.93405e-05,
    2.81001e-05,
    0.000179808,
    1.19114e-05,
    0.000101626,
    3.31354e-05,
    3.82789e-05,
    1.42937e-05,
    4.55341e-05,
    6.11272e-05,
    0.000238499,
    0.00018414,
    3.1457e-05,
    0.000300059,
    0.000108989,
    3.91452e-05,
    3.55718e-05,
    0.000182894,
    0.000232056,
    9.09599e-05,
    0.000162103,
    0.000326806,
    6.38885e-05,
    0.0002261,
    0.000563843,
    0.001324224,
    0.000740511,
    0.000310671,
    0.00073174,
    0.000258423,
    0.000180999,
    0.000429298,
    0.000315111,
    2.90205e-05,
    1.57014e-06,
    8.52749e-05,
    3.47597e-05,
    3.85497e-05,
    3.10238e-05,
    0,
]

In [593]:
if use_real_dataset:
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=X, xbins=dict(start=0, end=1, size=0.005), name="With linear modifier function"))
    fig.update_xaxes(range=[0, 1])
    fig.update_yaxes(range=[0, 1000])
    fig.show()